In [1]:
import pandas as pd
import sys

# 1. 파일 이름 정의
target_file = 'TEST_earthquakes_filtered_1_USGS.csv'
source_file = 'earthquakes_filtered_include_depth.csv'
output_file = 'TEST_earthquakes.csv' # 저장할 최종 파일 이름

try:
    # 2. 두 개의 CSV 파일 로드
    df_target = pd.read_csv(target_file)
    df_source = pd.read_csv(source_file)

    print(f"--- 원본 파일 '{target_file}' 로드 완료 ({len(df_target)} 행) ---")
    print("합치기 전 컬럼:", df_target.columns.tolist())

    print(f"\n--- 'depth' 정보 파일 '{source_file}' 로드 완료 ({len(df_source)} 행) ---")
    print("'depth' 파일 컬럼:", df_source.columns.tolist())

    # 3. 합치기 기준이 될 공통 컬럼 정의
    # (두 파일에 공통으로 존재하는 컬럼을 기준으로 함)
    merge_keys = ['tsunami', 'magnitude', 'latitude', 'longitude', 'Year', 'Month']

    # 4. 'df_source'에서 'depth'와 공통 키만 선택
    #    (혹시 모를 중복을 대비해 공통 키 기준으로 중복 제거)
    df_source_with_depth = df_source[merge_keys + ['depth']].drop_duplicates(subset=merge_keys)

    # 5. 'left' merge 실행
    # 'df_target'의 모든 행은 유지하고, 'df_source_with_depth'에서 키가 일치하는 'depth'를 가져옴
    print("\n--- 두 데이터 병합 작업 수행 (기준: 6개 키) ---")
    df_merged = pd.merge(df_target, df_source_with_depth, on=merge_keys, how='left')

    # 6. 병합 성공/실패 여부 확인
    successful_matches = df_merged['depth'].notna().sum()
    failed_matches = df_merged['depth'].isna().sum()

    print(f"\n--- 병합 결과 요약 ---")
    print(f"  - 'depth' 값을 성공적으로 찾아 합친 행: {successful_matches} 개")
    print(f"  - 일치하는 'depth' 값을 찾지 못한 행: {failed_matches} 개")

    if failed_matches > 0:
        print("\n[경고] 일부 행의 'depth' 값을 찾지 못했습니다.")
        print("     두 파일의 소수점 값이 정확히 일치하는지 확인해보세요.")

    # 7. 새 CSV 파일로 저장
    df_merged.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n[성공] 'depth'가 합쳐진 파일이 '{output_file}'으로 저장되었습니다.")
    print("최종 컬럼:", df_merged.columns.tolist())

except FileNotFoundError as e:
    print(f"[오류] 파일을 찾을 수 없습니다: {e.filename}", file=sys.stderr)
except KeyError as e:
    print(f"[오류] 필요한 기준 컬럼을 찾을 수 없습니다: {e}", file=sys.stderr)
    print("     두 파일에 'tsunami', 'magnitude', 'latitude', 'longitude', 'Year', 'Month'가 모두 있는지 확인하세요.", file=sys.stderr)
except Exception as e:
    print(f"[오류] 데이터를 처리하는 중 오류가 발생했습니다: {e}", file=sys.stderr)

--- 원본 파일 'TEST_earthquakes_filtered_1_USGS.csv' 로드 완료 (661 행) ---
합치기 전 컬럼: ['tsunami', 'magnitude', 'latitude', 'longitude', 'Year', 'Month', 'horizontal_count_1y_full', 'vertical_count_1y_full']

--- 'depth' 정보 파일 'earthquakes_filtered_include_depth.csv' 로드 완료 (661 행) ---
'depth' 파일 컬럼: ['tsunami', 'magnitude', 'depth', 'latitude', 'longitude', 'Year', 'Month']

--- 두 데이터 병합 작업 수행 (기준: 6개 키) ---

--- 병합 결과 요약 ---
  - 'depth' 값을 성공적으로 찾아 합친 행: 661 개
  - 일치하는 'depth' 값을 찾지 못한 행: 0 개

[성공] 'depth'가 합쳐진 파일이 'TEST_earthquakes.csv'으로 저장되었습니다.
최종 컬럼: ['tsunami', 'magnitude', 'latitude', 'longitude', 'Year', 'Month', 'horizontal_count_1y_full', 'vertical_count_1y_full', 'depth']
